# ANALYSIS 5 : RECOMMENDER SYSTEM FOR BRAND RUBIE'S COSTUME CO. 
- Collaborative filtering algorithms is used to get the recomendations.
- The Recommender System will take the 'Product Name' and based on the correlation factor will give output as list of products which will be a suggestion or recommendation.
- Suppose product name 'A' act as input parameter i.e. If a user buy product 'A' so based on that it will output the product highly correlated to it.
-----------------------------------------

Importing all the required Libraries

In [2]:
import glob
import json
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

Creating a path for Review file 'ReviewSample.json' which is the One of the input dataset for creating Recommender System.

In [5]:
files=glob.glob('../Data/Tested_Data/*')

Cleaning of 'ProductSample.json' file and importing the data as pandas DataFrame.

In [7]:
# Reading a multiple json files from a single json file 'ProductSample.json'.
product=[]
with open(files[0]) as data_file:
    data=data_file.read()
    for i in data.split('\n'):
        product.append(i)
        
# Firstly cleaning the data by converting files into proper json format files by some replacements and 
# then Making a list of Tuples containg data of brand Rubie's Costume Co.
productDataframe=[]
brand_List=['Rubie&#39;s Costume Co']
for x in product:
    try:
        y=x.replace("'",'"')
        jdata=json.loads(y)
        if jdata['brand'] in brand_List:
            productDataframe.append((jdata['asin'],jdata['title'])) 
    except:
        pass
    
# Creating a dataframe using the list of Tuples got in the previous step.  
Product_dataset=pd.DataFrame(productDataframe,columns=['Asin','Title'])

Getting all the distinct product Asin in list.  

In [8]:
prod_List=Product_dataset.Asin.unique().tolist()

Cleaning of 'ReviewSample.json' file and importing the data as pandas DataFrame.

In [9]:
# Reading a multiple json files from a single json file 'ReviewSample.json'.
review=[]
with open(files[1]) as data_file:
    data=data_file.read()
    for i in data.split('\n'):
        review.append(i)
        
# Making a list of Tuples containg all the data of json files.
reviewDataframe=[]
for x in review:
    try:
        jdata=json.loads(x)
        reviewDataframe.append((jdata['reviewerID'],jdata['asin'],jdata['overall'])) 
    except:
        pass
    
# Creating a dataframe using the list of Tuples got in the previous step.     
review_dataset=pd.DataFrame(reviewDataframe,columns=['Reviewer_ID','Asin','Rating'])    

Creating a DataFrame 'Working_dataset' which has products only from brand "RUBIE'S COSTUME CO."

In [10]:
Working_dataset=review_dataset[review_dataset.Asin.isin(prod_List)]

Performing a merge of 'Working_dataset' and 'Product_dataset' to get all the required details together for building the Recommender system.

In [11]:
Working_dataset=pd.merge(Working_dataset,Product_dataset,on='Asin',how='inner')

- Taking only the required columns and creating a pivot table with index as 'Reviewer_ID' , columns as 'Title' and values as 'Rating'.
- 'Model' is a pivot table.

In [13]:
Working_dataset=Working_dataset[['Reviewer_ID','Asin','Title','Rating']]
Model = Working_dataset.pivot_table(index='Reviewer_ID',columns='Title',values='Rating')

- Function to find the pearson correlation between two columns or products.
- Will produce result between -1 to 1.
- Function will be used within the recommender function 'get_recommendations()'.

In [14]:
def pearson(x1,x2):
    x1_cor=x1-x1.mean()
    x2_cor=x2-x2.mean()
    return np.sum(x1_cor * x2_cor)/np.sqrt(np.sum(x1_cor**2) * np.sum(x2_cor**2))

- Function to recommend the product based on correlation between them.
- Takes 3 parameters 'Product Name', 'Model' and 'Number of Recomendations'
- Will return a list in descending order of correlation and the list size depends on the input given for Number of Recomendations.

In [15]:
def get_recommendations(product_id,M,num):
    recomend=[]
    for asin in M.columns:
        if asin==product_id:
            continue
        cor=pearson(M[product_id],M[asin])
        if np.isnan(cor):
            continue
        else:
            recomend.append((asin,cor))
    recomend.sort(key=lambda tup: tup[1],reverse=True)
    return recomend[:num]

Function to replace all the html escape characters to english.

In [17]:
def escape(t):
    """HTML-escape the text in `t`."""
    return (t
        .replace("&amp;","&").replace("&lt;","<").replace("&gt;",">")
        .replace("&#39;","'").replace("&quot;",'"')
        )

- Calling the recommender System by making a function call to 'get_recommendations()'.
- '300 Movie Spartan Shield' is the product name pass to the function i.e. if person buys '300 Movie Spartan Shield' what else can be recommended to him/her.
- 'Model' is passed for correlation calculation.
- '5' is the maximum number of recommendation a function can return if there is some correlation.
- Quantifying the correlation can be done by using correlation value given in the output.

In [19]:
rec=get_recommendations('300 Movie Spartan Shield',Model,5)

- Taking recommendation into DataFrame for Tabular represtation.
- Takng only those values whose correlation is greater than 0.

In [20]:
Recommendation=[]
for x in rec:
    if x[1] > 0:
        Recommendation.append((escape(x[0]),x[1]))

result=pd.DataFrame(Recommendation,columns=['Product Title','Correlation '])    

Taking all the recommendations into .csv file

In [22]:
result.to_csv('../Analysis/Analysis_5/Recommendation.csv')

Displaying rows of resultant DataFrame

In [23]:
result

,Product Title,Correlation
0,300 Movie Spartan Deluxe Vinyl Helmet,0.223277
1,Toynk Toys - 300- Spartan Sword,0.069275
